# Writing Custom Metrics
Let's start by generating a simple synthetic dataset.

In [1]:
import warnings

warnings.filterwarnings("ignore")

In [2]:
from sdv import load_demo, SDV

sdv = SDV()
metadata, real_tables = load_demo(metadata=True)

sdv.fit(metadata, real_tables)
synthetic_tables = sdv.sample_all(20)

2020-06-25 19:47:19,922 - INFO - modeler - Modeling users
2020-06-25 19:47:19,923 - INFO - metadata - Loading transformer CategoricalTransformer for field country
2020-06-25 19:47:19,924 - INFO - metadata - Loading transformer CategoricalTransformer for field gender
2020-06-25 19:47:19,924 - INFO - metadata - Loading transformer NumericalTransformer for field age
2020-06-25 19:47:19,941 - INFO - modeler - Modeling sessions
2020-06-25 19:47:19,942 - INFO - metadata - Loading transformer CategoricalTransformer for field device
2020-06-25 19:47:19,942 - INFO - metadata - Loading transformer CategoricalTransformer for field os
2020-06-25 19:47:19,954 - INFO - modeler - Modeling transactions
2020-06-25 19:47:19,955 - INFO - metadata - Loading transformer DatetimeTransformer for field timestamp
2020-06-25 19:47:19,955 - INFO - metadata - Loading transformer NumericalTransformer for field amount
2020-06-25 19:47:19,955 - INFO - metadata - Loading transformer BooleanTransformer for field appro

Next, we'll create an empty `MetricsReport` object to hold our custom metrics.

In [3]:
from sdmetrics.report import MetricsReport

report = MetricsReport()

## Generic Metric
The simplest way to create a custom metric is to use the generic metric API. You simply write a function which yields a sequence of Metric objects, attach it to a metrics report, and you're ready to go!

In [4]:
from sdmetrics.report import Metric

def my_custom_metrics(metadata, real_tables, synthetic_tables):
    name = "abs-diff-in-number-of-rows"

    for table_name in real_tables:

        # Absolute difference in number of rows
        nb_real_rows = len(real_tables[table_name])
        nb_synthetic_rows = len(synthetic_tables[table_name])
        value = float(abs(nb_real_rows - nb_synthetic_rows))

        # Specify some useful tags for the user
        tags = set([
            "priority:high",
            "table:%s" % table_name
        ])

        yield Metric(name, value, tags)
        
report.add_metrics(my_custom_metrics(metadata, real_tables, synthetic_tables))

## Statistic Metric
Alternatively, if you're looking to create a statistical metric which looks at univariate or bivariate distributions, you can subclass the `UnivariateMetric` class and fill in a single function. The base class will handle identifying the columns which have the correct data type, traversing the tables, and so on. You can simply focus on the math.

In [5]:
from scipy.stats import chisquare

from sdmetrics.report import Goal
from sdmetrics.statistical.univariate import UnivariateMetric
from sdmetrics.statistical.utils import frequencies

class CSTest(UnivariateMetric):

    name = "chisquare"
    dtypes = ["object", "bool"]

    @staticmethod
    def metric(real_column, synthetic_column):
        """This function uses the Chi-squared test to compare the distributions
        of the two categorical columns. It returns the resulting p-value so that
        a small value indicates that we can reject the null hypothesis (i.e. and
        suggests that the distributions are different).

        Arguments:
            real_column (np.ndarray): The values from the real database.
            synthetic_column (np.ndarray): The values from the fake database.

        Returns:
            (str, Goal, str, tuple): A tuple containing (value, goal, unit, domain)
            which corresponds to the fields in a Metric object.
        """
        f_obs, f_exp = frequencies(real_column, synthetic_column)
        statistic, pvalue = chisquare(f_obs, f_exp)
        return pvalue, Goal.MAXIMIZE, "p-value", (0.0, 1.0)

report.add_metrics(CSTest().metrics(metadata, real_tables, synthetic_tables))

## Detection Metric
Similarly, if you're looking to create a detection metric, you can subclass the `TabularDetector` class and fill in the `fit` and `predict_proba` functions. The base class will handle denormalizing parent-child relationships, etc. so you can focus on the machine learning.

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.svm import SVC

from sdmetrics.detection.tabular import TabularDetector


class SVCDetector(TabularDetector):

    name = "svc"

    def fit(self, X, y):
        """This function trains a sklearn pipeline with a robust scalar
        and a support vector classifier.

        Arguments:
            X (np.ndarray): The numerical features (i.e. transformed rows).
            y (np.ndarray): The binary classification target.
        """
        self.model = Pipeline([
            ('scalar', RobustScaler()),
            ('classifier', SVC(probability=True, gamma='scale')),
        ])
        self.model.fit(X, y)

    def predict_proba(self, X):
        return self.model.predict_proba(X)[:, 1]

report.add_metrics(SVCDetector().metrics(metadata, real_tables, synthetic_tables))

## Collecting Metrics
Now that we've generated all the metrics, we can explore the value of each metric using the standard `MetricsReport` interface which allows users to summarize, visualize, and explore the metrics at various levels of granularity.

In [7]:
report.details()

,Columns,Goal,Misc. Tags,Name,Tables,Unit,Value
0,,Goal.IGNORE,priority:high,abs-diff-in-number-of-rows,table:users,,10.000000
1,,Goal.IGNORE,priority:high,abs-diff-in-number-of-rows,table:sessions,,9.000000
2,,Goal.IGNORE,priority:high,abs-diff-in-number-of-rows,table:transactions,,171.000000
3,column:country,Goal.MAXIMIZE,statistic:univariate,chisquare,table:users,p-value,0.980379
4,column:gender,Goal.MAXIMIZE,"priority:high,statistic:univariate",chisquare,table:users,p-value,0.000000
5,column:device,Goal.MAXIMIZE,statistic:univariate,chisquare,table:sessions,p-value,0.880443
6,column:os,Goal.MAXIMIZE,statistic:univariate,chisquare,table:sessions,p-value,0.779993
7,column:approved,Goal.MAXIMIZE,statistic:univariate,chisquare,table:transactions,p-value,0.778779
8,,Goal.MINIMIZE,detection:auroc,svc,table:users,auroc,0.716270
9,,Goal.MINIMIZE,detection:auroc,svc,table:transactions,auroc,0.962614
